In [ ]:
import sys
#print(sys.version)

import matplotlib.pyplot as plt
#%matplotlib inline
#%config BackendInline.figure_format='svg'

plt.style.use(['seaborn-paper', 'seaborn-white', 'seaborn-ticks'])
#plt.matplotlib.rcParams['figure.dpi'] = 120
#plt.matplotlib.rcParams['savefig.dpi'] = 120

import numpy as np
import pandas as pd

#import mlgrad.model as model
import mlgrad.func as func
import mlgrad.distance as distance
#import mlgrad.regular as regular
#import mlgrad.loss as loss
import mlgrad.weights as weights
import mlgrad.averager as averager

import mlgrad.avragg as avragg
import mlgrad.mlocation_scatter2 as mlocation_scatter

from mlgrad import averager_it, averager_fg, fg, erm_fg, sg, erm_sg, irgd, erm_irgd, drisk

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
print(digits.data.shape, digits.data.flags)

Y = digits.target
X = np.ascontiguousarray(digits.data)

In [ ]:
arith = avragg.ArithMean()
km = mlocation_scatter.MLocationsScattersEstimator(arith, 10, 
                                    tol=1.0e-9, n_iter=1000, n_step=22)
km.fit(X)

In [ ]:
fig, ax = plt.subplots(2, 5, figsize=(8, 3))
centers = km.locs.base.reshape(10, 8, 8)
for axi, center in zip(ax.flat, centers):
    axi.set(xticks=[], yticks=[])
    axi.imshow(center, interpolation='nearest', cmap=plt.cm.binary)

In [ ]:
from scipy.stats import mode

hcd_sq_labels = np.zeros_like(hcd_sq.Y.base)
for i in range(10):
    mask = (hcd_sq.Y.base == i)
    hcd_sq_labels[mask] = mode(Y[mask])[0]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Y, hcd_sq_labels)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
mat = confusion_matrix(Y, hcd_sq_labels)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=digits.target_names,
            yticklabels=digits.target_names)
plt.xlabel('true label')
plt.ylabel('predicted label')
plt.show()

In [ ]:
avrfunc = averager_it(func.QuantileFunc(0.9, func.Sqrt(0.001)))
avrfunc = avragg.ParameterizedAverage(func.WinsorizedSmoothFunc(func.Sqrt(0.001)), avrfunc)
hcd = mlocation_scatter.MLocationsScattersEstimator(avrfunc, 10, 
                                    tol=1.0e-9, n_iter=500, n_step=30)
hcd.fit(X)

In [ ]:
fig, ax = plt.subplots(2, 5, figsize=(8, 3))
centers = hcd.locs.base.reshape(10, 8, 8)
for axi, center in zip(ax.flat, centers):
    axi.set(xticks=[], yticks=[])
    axi.imshow(center, interpolation='nearest', cmap=plt.cm.binary)

In [ ]:
from scipy.stats import mode

hcd_rob_labels = np.zeros_like(hcd_rob.Y.base)
for i in range(10):
    mask = (hcd.Y.base == i)
    hcd_rob_labels[mask] = mode(Y[mask])[0]


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
mat = confusion_matrix(Y, hcd_rob_labels)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=digits.target_names,
            yticklabels=digits.target_names)
plt.xlabel('true label')
plt.ylabel('predicted label')
plt.show()